## Tuning Disparity Map ##
<b>Description:</b> This code will load the stored Stereo Calibration data and then provde you with sliders where you can tune and adjust your resulting disparity image.
<hr>

#### Cell Block 1 ####
<b>Description:</b> This cell block just imports all the necessary libraries, loads up the left/right image pair, and loads up the stored calibration data

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
from stereovision.calibration import StereoCalibrator
from stereovision.calibration import StereoCalibration
from stereovision.stereo_cameras import CalibratedPair

# Global variables preset
directory = "images"
photo_width = 640
photo_height = 480
image_width = 640
image_height = 480
image_size = (image_width,image_height)

image_pathL = os.path.join(directory, '14_imageLeft.jpg')
image_pathR = os.path.join(directory, '14_imageRight.jpg')

if os.path.isfile(image_pathL) == False:
    print ('Can not read image from file \"'+ image_pathL +'\"')
    exit(0)

# Read image and split it in a stereo pair
imgLeft = cv2.imread(image_pathL, 0)
imgRight = cv2.imread(image_pathR, 0)

# Implementing calibration data
print('Read calibration data and rectifying stereo pair...')
calibration = StereoCalibration(input_folder='calib_result')
rectified_pair = calibration.rectify((imgLeft, imgRight))

print('done')

#### Cell Block 2/3 ####
<b>Description:</b> Verify the calculated rectified pair from the calibration data. If the images here look incorrect, then you should redo your calibration dataset and recalibrate.

In [ ]:
plt.imshow(rectified_pair[1])
plt.show()

In [ ]:
plt.imshow(rectified_pair[0])
plt.show()

#### Cell Block 4 ####
<b>Description:</b> This code initializes the depth map with default values. Feel free to change the values and experiment with initial image.

In [ ]:
# Depth map function
SWS = 5
PFS = 5
PFC = 29
MDS = -25
NOD = 128
TTH = 100
UR = 10
SR = 15
SPWS = 100

def stereo_depth_map(rectified_pair):
    print ('SWS='+str(SWS)+' PFS='+str(PFS)+' PFC='+str(PFC)+' MDS='+\
           str(MDS)+' NOD='+str(NOD)+' TTH='+str(TTH))
    print (' UR='+str(UR)+' SR='+str(SR)+' SPWS='+str(SPWS))
    c, r = rectified_pair[0].shape
    disparity = np.zeros((c, r), np.uint8)
    sbm = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    sbm.setPreFilterType(1)
    sbm.setPreFilterSize(PFS)
    sbm.setPreFilterCap(PFC)
    sbm.setMinDisparity(MDS)
    sbm.setNumDisparities(NOD)
    sbm.setTextureThreshold(TTH)
    sbm.setUniquenessRatio(UR)
    sbm.setSpeckleRange(SR)
    sbm.setSpeckleWindowSize(SPWS)
    dmLeft = rectified_pair[0]
    dmRight = rectified_pair[1]
    disparity = sbm.compute(dmLeft, dmRight)
    local_max = disparity.max()
    local_min = disparity.min()
    print ("MAX " + str(local_max))
    print ("MIN " + str(local_min))
    disparity_grayscale = (disparity-local_min)*(65535.0/(local_max-local_min))
    disparity_fixtype = cv2.convertScaleAbs(disparity_grayscale, alpha=(255.0/65535.0))
    disparity_color = cv2.applyColorMap(disparity_fixtype, cv2.COLORMAP_JET)
    print ("MAX " + str(local_max))
    print ("MIN " + str(local_min))
    return disparity_color

disparity = stereo_depth_map(rectified_pair)
flipDisparity = cv2.cvtColor(disparity, cv2.COLOR_BGR2RGB)
print(type(disparity))

#### Cell Block 5 ####
<b>Description:</b> After running below code, you will see dynamic sliders for each variable that affects the depth map. Use this to find what values make the best depth map.

In [ ]:
spklWinSizeW = widgets.FloatSlider(value=100, min=0, max=300, step=0.1, description='Spkl Win Size:', disabled=False, continuous_update=True)
speckleRangeW = widgets.FloatSlider(value=15, min=0, max=40, step=0.1, description='Spkl Range:', disabled=False, continuous_update=True)
uniquenessRatioW = widgets.FloatSlider(value=10.0, min=1, max=20, step=0.1, description='Unique Ratio:', disabled=False, continuous_update=True)
textureThresholdW = widgets.FloatSlider(value=100, min=0, max=1000, step=0.1, description='Txtr Thresh:', disabled=False, continuous_update=True)
numDisparitiesW = widgets.FloatSlider(value=128, min=16, max=256, step=0.1, description='Num Disp:', disabled=False, continuous_update=True)
minDisparityW = widgets.FloatSlider(value=-25, min=-100, max=100, step=0.1, description='Min Disp:', disabled=False, continuous_update=True)
preFilterCapW = widgets.FloatSlider(value=29, min=5, max=63, step=0.1, description='PreFilt Cap:', disabled=False, continuous_update=True)
preFilterSizeW = widgets.FloatSlider(value=5, min=5, max=255, step=0.1, description='PreFilt Size:', disabled=False, continuous_update=True)
SADWindowSizeW = widgets.FloatSlider(value=5, min=5, max=255, step=0.1, description='SAD Win Size:', disabled=False, continuous_update=True)

output2 = widgets.Output()
display(spklWinSizeW, speckleRangeW, uniquenessRatioW, textureThresholdW, numDisparitiesW, minDisparityW, preFilterCapW, preFilterSizeW, SADWindowSizeW, output2)

def updateVal(change):
    global SWS, PFS, PFC, MDS, NOD, TTH, UR, SR, SPWS, disparity
    with output2:
        clear_output()
        SWS = int(SADWindowSizeW.value/2)*2+1
        PFS = int(preFilterSizeW.value/2)*2+1
        PFC = int(preFilterCapW.value/2)*2+1    
        MDS = int(minDisparityW.value)    
        NOD = int(numDisparitiesW.value/16)*16  
        TTH = int(textureThresholdW.value)
        UR = int(uniquenessRatioW.value)
        SR = int(speckleRangeW.value)
        SPWS= int(spklWinSizeW.value)
        disparity = stereo_depth_map(rectified_pair)
        flipDisparity = cv2.cvtColor(disparity, cv2.COLOR_BGR2RGB)
        plt.imshow(flipDisparity)
        plt.show()

spklWinSizeW.observe(updateVal, names='value')
speckleRangeW.observe(updateVal, names='value')
uniquenessRatioW.observe(updateVal, names='value')
textureThresholdW.observe(updateVal, names='value')
numDisparitiesW.observe(updateVal, names='value')
minDisparityW.observe(updateVal, names='value')
preFilterCapW.observe(updateVal, names='value')
preFilterSizeW.observe(updateVal, names='value')
SADWindowSizeW.observe(updateVal, names='value')


